In [1]:
!pip install accelerate peft datasets bitsandbytes evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 21.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00


In [2]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4e1d18159e8eb1d871bf02edc15d8fc3dd9b7d9a87f7f84b330785d633431c9a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


# **Import library and load module**

In [8]:
import evaluate
import torch
import json
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    default_data_collator,
)
from datasets import load_dataset, Dataset, DatasetDict
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from tqdm import tqdm
import os

In [9]:
from huggingface_hub import login

hf_token = os.getenv("HF_TOKEN")

login(token=hf_token)

In [10]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# **Continue Pretrain**

In [6]:
model_id = "meta-llama/Llama-3.2-1B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
)

# 4. QLoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.config.use_cache = False

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("khai2k48/vietnamese_news_17k", split="train")
dataset = dataset.select(range(2000))

In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(examples["content"], padding="max_length", truncation=True, max_length=2048)
    labels = tokenized["input_ids"].copy()
    labels = [-100 if token == tokenizer.pad_token_id else token for token in labels]
    tokenized["labels"] = labels
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

train_loader = DataLoader(tokenized_dataset, batch_size=2, shuffle=True)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Số epoch
n_epochs = 3

for epoch in range(n_epochs):
    model.train()
    total_loss = 0

    # Thêm tqdm để hiển thị tiến trình training
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}", leave=True)

    for batch in progress_bar:
        # Đưa batch vào GPU
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backpropagation và cập nhật trọng số
        optimizer.zero_grad()
        loss.backward()

        # Gradient Clipping (tránh exploding gradients)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())  # Hiển thị loss từng batch trên tqdm

    # Tính loss trung bình của epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {avg_loss:.4f}")


Epoch 1/3:   0%|          | 0/1000 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Epoch 1/3: 100%|██████████| 1000/1000 [2:57:23<00:00, 10.64s/it, loss=0.828] 


Epoch 1/3 - Loss: 1.2615


Epoch 2/3: 100%|██████████| 1000/1000 [2:57:27<00:00, 10.65s/it, loss=1.33]  


Epoch 2/3 - Loss: 1.1460


Epoch 3/3: 100%|██████████| 1000/1000 [2:57:27<00:00, 10.65s/it, loss=0.825] 

Epoch 3/3 - Loss: 1.1380


In [10]:
output_dir = "/kaggle/working/llama3-qlora-continued-pretrain"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Pretrained model saved at:", output_dir)

✅ Pretrained model saved at: /kaggle/working/llama3-qlora-continue-pretrained


# **Finetune with QA dataset**

In [11]:
model_id = "/kaggle/input/llama-qa-partially-finetune/transformers/default/1/llama-qa-finetune"

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
)

# 4. QLoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.config.use_cache = False

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [13]:
with open("/kaggle/input/vietnamese-squad/train-v2.0-translated.json", "r", encoding="utf-8") as f:
    vi_squad_data = json.load(f)

In [14]:
train_data = vi_squad_data[:10000]

# Lấy 2k mẫu tiếp theo cho validation
val_data = vi_squad_data[10000:12000]

print(f"📦 Train samples: {len(train_data)}")
print(f"📦 Validation samples: {len(val_data)}")

📦 Train samples: 10000
📦 Validation samples: 2000


In [15]:
print(f"📦 Train samples: {train_data[0]}")

📦 Train samples: ['Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên ở Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny\'s Child. Được quản lý bởi cha cô, Mathew Knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 "Crazy in Love" và "Baby Boy".', 'Beyonce bắt đầu nổi tiếng từ khi nào?', 'Vào cuối những năm 1990']


In [16]:
train_dataset = Dataset.from_list([{"context": item[0], "question": item[1], "answer": item[2]} for item in train_data])
val_dataset = Dataset.from_list([{"context": item[0], "question": item[1], "answer": item[2]} for item in val_data])

dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": val_dataset
})

In [17]:
print(f"1. {dataset_dict['train'][0]}")
print(f"2. {dataset_dict['train'][0]['context']}")
print(f"3. {dataset_dict['train'][0]['question']}")
print(f"4. {dataset_dict['train'][0]['answer']}")

1. {'context': 'Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên ở Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny\'s Child. Được quản lý bởi cha cô, Mathew Knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 "Crazy in Love" và "Baby Boy".', 'question': 'Beyonce bắt đầu nổi tiếng từ khi nào?', 'answer': 'Vào cuối những năm 1990'}
2. Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản 

In [18]:
def preprocess_qa(example, max_length=512):
    context = example["context"]
    question = example["question"]
    answer = example["answer"]
    prompt = f"### Đây là dạng câu hỏi và trả lời dựa trên nội dung ###\n\nCâu hỏi: {question}\n\nNội dung: {context}\n\nTrả lời:"
    completion = f" {answer}"
    
    prompt_ids = tokenizer(prompt, add_special_tokens=False).input_ids
    completion_ids = tokenizer(completion, add_special_tokens=False).input_ids

    input_ids = prompt_ids + completion_ids

    labels = [-100] * len(prompt_ids) + completion_ids

    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
        labels = labels[:max_length]

    attention_mask = [1] * len(input_ids)
    padding_length = max_length - len(input_ids)
    
    input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
    attention_mask = attention_mask + [0] * padding_length
    labels = labels + [-100] * padding_length
    
    return {
        "input_ids": torch.tensor(input_ids),
        "attention_mask": torch.tensor(attention_mask),
        "labels": torch.tensor(labels)
    }

tokenized_dataset = dataset_dict.map(preprocess_qa)

tokenized_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [19]:
collator = default_data_collator

train_loader = DataLoader(
    tokenized_dataset["train"], batch_size=4, shuffle=True, collate_fn=collator
)

val_loader = DataLoader(
    tokenized_dataset["test"], batch_size=4, shuffle=False, collate_fn=collator
)

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

n_epochs = 2

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}", leave=True)

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1}/{n_epochs} - Train Loss: {avg_loss:.4f}")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"📊 Validation Loss: {avg_val_loss:.4f}")


Epoch 1/2:   0%|          | 0/2500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Epoch 1/2: 100%|██████████| 2500/2500 [3:15:37<00:00,  4.69s/it, loss=0.853]  


✅ Epoch 1/2 - Train Loss: 1.6862
📊 Validation Loss: 1.4126


Epoch 2/2: 100%|██████████| 2500/2500 [3:15:50<00:00,  4.70s/it, loss=0.749]  


✅ Epoch 2/2 - Train Loss: 1.3846
📊 Validation Loss: 1.2764


In [27]:
output_dir = "/kaggle/working/llama3-qlora-qa-finetune"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Pretrained model saved at:", output_dir)

✅ Pretrained model saved at: /kaggle/working/llama3-qlora-qa-finetune


In [35]:
import shutil
from IPython.display import FileLink

# Đường dẫn tới thư mục chứa mô hình
model_dir = '/kaggle/working/llama3-qlora-qa-finetune'

# Tạo file nén .zip từ thư mục
zip_name = 'llama3-qlora-qa-finetune'  # Tên file nén (không cần đuôi .zip)
shutil.make_archive(zip_name, 'zip', model_dir)

# Tạo liên kết tải xuống cho file .zip
display(FileLink(f'{zip_name}.zip'))

/kaggle/working/llama3-qlora-qa-finetune.zip

# **Finetune with BBC News**

In [6]:
model_id = "/kaggle/input/llama-qa-finetune/transformers/default/1"

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
)

# 4. QLoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.config.use_cache = False

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [12]:
model.gradient_checkpointing_enable()

In [14]:
data_path = "/kaggle/input/data-llama-finetune/bbc_data_llama_finetune.json"

full_dataset = load_dataset("json", data_files=data_path, split="train")
dataset = full_dataset.train_test_split(test_size=0.1)

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
def tokenize(example, max_length=2048):
    prompt = f"### Đây là dạng tóm tắt văn bản tin tức với độ dài tóm tắt đầu ra khoảng 150 từ:  ### Prompt:\n{example['prompt']}\n\n### Summary:\n"
    summary = example["summary"]
    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    summary_ids = tokenizer.encode(summary, add_special_tokens=False)
    
    # Kiểm tra tổng số token
    total_length = len(prompt_ids) + len(summary_ids)
    if total_length > max_length:
        overflow = total_length - max_length
        if overflow < len(prompt_ids):
            prompt_ids = prompt_ids[:-overflow]
        else:
            prompt_ids = []
    input_ids = prompt_ids + summary_ids
    attention_mask = [1] * len(input_ids)
    labels = [-100] * len(prompt_ids) + summary_ids

    padding_length = max_length - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
        attention_mask = attention_mask + [0] * padding_length
        labels = labels + [-100] * padding_length
    else:
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }
tokenized_dataset = dataset.map(tokenize, batched=False, fn_kwargs={"max_length": 2048})
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

collator = default_data_collator

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=2,
    shuffle=True,
    collate_fn=collator
)

val_loader = DataLoader(
    tokenized_dataset["test"],
    batch_size=2,
    shuffle=False,
    collate_fn=collator
)


Map:   0%|          | 0/1914 [00:00<?, ? examples/s]

Map:   0%|          | 0/213 [00:00<?, ? examples/s]

In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [17]:
n_epochs = 3

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    model.train()
    pbar = tqdm(train_loader)
    total_loss = 0

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_description(f"Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} completed — Avg loss: {avg_loss:.4f}")

Epoch 1/3


Loss: 0.1446: 100%|██████████| 957/957 [2:49:01<00:00, 10.60s/it]  


✅ Epoch 1 completed — Avg loss: 0.1809
Epoch 2/3


Loss: 0.1074: 100%|██████████| 957/957 [2:49:16<00:00, 10.61s/it]  


✅ Epoch 2 completed — Avg loss: 0.1497
Epoch 3/3


Loss: 0.1099: 100%|██████████| 957/957 [2:49:09<00:00, 10.61s/it]  

✅ Epoch 3 completed — Avg loss: 0.1431


In [18]:
output_dir = "/kaggle/working/llama3-qlora-bbc-news-finetuned"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Finetuned model saved at:", output_dir)

✅ Finetuned model saved at: /kaggle/working/llama3-qlora-finetuned


In [19]:
def evaluate_after_training(model, val_loader, tokenizer, device, num_samples=3):
    import evaluate
    from torch.nn.utils.rnn import pad_sequence

    model.eval()
    predictions = []
    references = []
    samples_to_print = []

    # Load ROUGE
    rouge = evaluate.load("rouge")

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    def left_pad(inputs, pad_token_id):
        """Chuyển batch input thành left-padded"""
        max_len = max(len(seq) for seq in inputs)
        return torch.stack([
            torch.cat([torch.full((max_len - len(seq),), pad_token_id, dtype=torch.long), seq])
            for seq in inputs
        ])

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            labels = batch["labels"]

            # Chuyển sang left padding thủ công
            input_ids = left_pad([x[x != tokenizer.pad_token_id] for x in input_ids], tokenizer.pad_token_id).to(device)
            attention_mask = (input_ids != tokenizer.pad_token_id).long()

            # Generate
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=128,
                num_beams=4,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

            # Decode
            generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

            label_texts = []
            for label in labels:
                label_ids = [token_id for token_id in label.tolist() if token_id != -100]
                label_texts.append(tokenizer.decode(label_ids, skip_special_tokens=True))

            predictions.extend(generated_texts)
            references.extend(label_texts)

            if len(samples_to_print) < num_samples:
                src_texts = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
                for src, ref, pred in zip(src_texts, label_texts, generated_texts):
                    samples_to_print.append((src, ref, pred))
                    if len(samples_to_print) >= num_samples:
                        break

    # Tính ROUGE
    results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    print("\n📊 ROUGE Scores:")
    for key in results:
        print(f"{key}: {results[key]:.4f}")

    print("\n📝 Sample Results:")
    for i, (src, ref, pred) in enumerate(samples_to_print):
        print(f"\n--- Sample {i+1} ---")
        print(f"[Prompt]    {src}")
        print(f"[Reference] {ref}")
        print(f"[Generated] {pred}")


In [20]:
evaluate_after_training(model, val_loader, tokenizer, device)

Evaluating:   0%|          | 0/107 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Evaluating: 100%|██████████| 107/107 [30:23<00:00, 17.04s/it]



📊 ROUGE Scores:
rouge1: 0.4069
rouge2: 0.4053
rougeL: 0.4071
rougeLsum: 0.4072

📝 Sample Results:

--- Sample 1 ---
[Prompt]    ### Đây là dạng tóm tắt văn bản tin tức với độ dài tóm tắt đầu ra khoảng 150 từ:  ### Prompt:
You are a helpful summarization assistant. Summarize the following article in 150 words: Campaign 'cold calls' questioned

Labour and the Conservatives are still telephoning the millions of people who have signed up to make sure they do not get marketing "cold calls".

The parties say they can stick to the rules by ensuring that their calls are not marketing - for instance by asking about people's voting intentions. The Lib Dems are asking the watchdog overseeing the rules to stop the calls. The information commissioner's office says surveys are allowed but people had to be told if personal data was kept. Telephone call centres are expected to be used as never before by all the three major parties in the run-up to the general election.

But seven million telephone nu

# **Finetune with donvanban**

In [29]:
model_id = "/kaggle/working/llama3-qlora-finetuned"

In [30]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
)

# 4. QLoRA config
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.gradient_checkpointing_enable()

In [31]:
data_path = "/kaggle/input/data-llama-finetune/vi_data_llama_finetune.json"

full_dataset = load_dataset("json", data_files=data_path, split="train")
dataset = full_dataset.train_test_split(test_size=0.1)

In [32]:
def tokenize(example, max_length=2048):
    prompt = f"### Đây là dạng tóm tắt văn bản tin tức với độ dài tóm tắt đầu ra khoảng 150 từ:  ### Lệnh:\n{example['prompt']}\n\n### Tóm tắt:\n"
    summary = example["summary"]
    
    # Mã hóa riêng prompt và summary (không thêm special tokens)
    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    summary_ids = tokenizer.encode(summary, add_special_tokens=False)
    
    # Kiểm tra tổng số token
    total_length = len(prompt_ids) + len(summary_ids)
    if total_length > max_length:
        overflow = total_length - max_length
        # Ưu tiên giữ lại phần summary; cắt bớt prompt
        if overflow < len(prompt_ids):
            prompt_ids = prompt_ids[:-overflow]
        else:
            prompt_ids = []  # Nếu quá tràn, bỏ hết prompt
    
    # Nối prompt và summary
    input_ids = prompt_ids + summary_ids
    attention_mask = [1] * len(input_ids)
    
    # Tạo labels: phần prompt được mask bằng -100, phần summary giữ nguyên token IDs
    labels = [-100] * len(prompt_ids) + summary_ids
    
    # Padding tất cả các trường về độ dài max_length
    padding_length = max_length - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
        attention_mask = attention_mask + [0] * padding_length
        labels = labels + [-100] * padding_length
    else:
        # Nếu quá dài, cắt bớt (nên không xảy ra nhờ truncation ở trên)
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }
tokenized_dataset = dataset.map(tokenize, batched=False, fn_kwargs={"max_length": 2048})
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

collator = default_data_collator

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=2,
    shuffle=True,
    collate_fn=collator
)

val_loader = DataLoader(
    tokenized_dataset["test"],
    batch_size=2,
    shuffle=False,
    collate_fn=collator
)


Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [33]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): L

In [35]:
n_epochs = 5

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    model.train()
    pbar = tqdm(train_loader)
    total_loss = 0

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_description(f"Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} completed — Avg loss: {avg_loss:.4f}")

Epoch 1/5


Loss: 0.3921: 100%|██████████| 72/72 [12:51<00:00, 10.72s/it]


✅ Epoch 1 completed — Avg loss: 0.5270
Epoch 2/5


Loss: 0.4752: 100%|██████████| 72/72 [12:48<00:00, 10.68s/it]


✅ Epoch 2 completed — Avg loss: 0.4871
Epoch 3/5


...

✅ Epoch 3 completed — Avg loss: 0.4527
Epoch 4/5


Loss: 0.2761: 100%|██████████| 72/72 [12:51<00:00, 10.71s/it]


✅ Epoch 4 completed — Avg loss: 0.4381
Epoch 5/5


Loss: 0.3524: 100%|██████████| 72/72 [12:50<00:00, 10.70s/it]

✅ Epoch 5 completed — Avg loss: 0.4144


In [36]:
output_dir = "/kaggle/working/llama3-qlora-finetuned-all-1"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Finetuned model saved at:", output_dir)

✅ Finetuned model saved at: /kaggle/working/llama3-qlora-finetuned-all-1


In [39]:
def evaluate_after_training(model, val_loader, tokenizer, device, num_samples=3):
    import evaluate
    from torch.nn.utils.rnn import pad_sequence

    model.eval()
    predictions = []
    references = []
    samples_to_print = []

    # Load ROUGE
    rouge = evaluate.load("rouge")

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    def left_pad(inputs, pad_token_id):
        """Chuyển batch input thành left-padded"""
        max_len = max(len(seq) for seq in inputs)
        return torch.stack([
            torch.cat([torch.full((max_len - len(seq),), pad_token_id, dtype=torch.long), seq])
            for seq in inputs
        ])

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            labels = batch["labels"]

            # Chuyển sang left padding thủ công
            input_ids = left_pad([x[x != tokenizer.pad_token_id] for x in input_ids], tokenizer.pad_token_id).to(device)
            attention_mask = (input_ids != tokenizer.pad_token_id).long()

            # Generate
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=128,
                num_beams=4,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

            # Decode
            generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

            label_texts = []
            for label in labels:
                label_ids = [token_id for token_id in label.tolist() if token_id != -100]
                label_texts.append(tokenizer.decode(label_ids, skip_special_tokens=True))

            predictions.extend(generated_texts)
            references.extend(label_texts)

            if len(samples_to_print) < num_samples:
                src_texts = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
                for src, ref, pred in zip(src_texts, label_texts, generated_texts):
                    samples_to_print.append((src, ref, pred))
                    if len(samples_to_print) >= num_samples:
                        break

    # Tính ROUGE
    results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    print("\n📊 ROUGE Scores:")
    for key in results:
        print(f"{key}: {results[key]:.4f}")

    print("\n📝 Sample Results:")
    for i, (src, ref, pred) in enumerate(samples_to_print):
        print(f"\n--- Sample {i+1} ---")
        print(f"[Prompt]    {src}")
        print(f"[Reference] {ref}")
        print(f"[Generated] {pred}")


In [40]:
evaluate_after_training(model, val_loader, tokenizer, device)

Evaluating: 100%|██████████| 9/9 [03:34<00:00, 23.87s/it]



📊 ROUGE Scores:
rouge1: 0.2992
rouge2: 0.2967
rougeL: 0.2985
rougeLsum: 0.2977

📝 Sample Results:

--- Sample 1 ---
[Prompt]    ### Đây là dạng tóm tắt văn bản tin tức với độ dài tóm tắt đầu ra khoảng 150 từ:  ### Lệnh:
Bạn là một trợ lý tóm tắt văn bản. Hãy cung cấp bản tóm tắt ngắn gọn và chính xác trong 150 chữ cho bài viết sau. Bài viết: ﻿Tạp chí Mỹ bị lên án vì chụp ảnh "thời trang tự tử"
 Tạp chí Vice của Mỹ vừa bị lên án vì thực hiện bộ ảnh "thời trang tự tử", tái hiện lại giây phút cuối đời của những nữ văn sĩ nổi tiếng. Trong những năm gần đây, để cạnh tranh với những báo khác, các tờ tạp chí thường mạnh dạn đưa ra những ý tưởng mới lạ, nhiều khi trở thành quá đà.  Đối với ý tưởng sáng tạo "thảm họa" lần này của tạp chí Vice, tờ Huffington Post đánh giá rằng: "Vice nghĩ họ đang tạo ra những bức ảnh nghệ thuật nhưng cuối cùng người ta chỉ thấy đó là những bức ảnh thiếu suy nghĩ và phản cảm". Sau khi những bức hình tự tử này bị xóa đi, Vice đã đăng tải lời xin lỗi: "Những bức ả